<a href="https://colab.research.google.com/github/its-me-piyush/Suggest-relevant-products-for-a-store/blob/main/Suggest_relevant_products_for_a_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem 1 - Suggest relevant products for a store**
## Problem Description - 

- Develop a ML system to determine relevant products for a store based on its past transaction history. These suggestions can be used in multiple sales channels to the store like self ordering app, sales executive PDA or by a calling agent to ensure that store is not missing out on any of these relevant products.

## Train a model on the given dataset to predict each customer's next order

- Participants may make the following assumptions

- Every store has max 1 transaction per day which may contain multiple products
All stores served from the same fulfillment center are exposed to the same set of products. Hence, participants may create separate models per fulfillment center.


# $Team: Adagrad$
## $Members:$
*   $Piyush Nagpal$
*   $Jay Raviraj Deore$
*   $Sarvesha Jaiswal$


<a id="importing-relevant-python-packages"></a>
## Importing Relevant Python Packages

In [ ]:
# importing necessary Python libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
#import plotly.offline as pyoff
import plotly.graph_objs as go 
#import plotly.figure_factory as ff

# avoid displaying warnings
import warnings
warnings.filterwarnings("ignore")

#import machine learning related libraries
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from multiscorer import MultiScorer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.cluster import KMeans
import xgboost as xgb
import time

In [ ]:
# Connect to gDrive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# unzip the dataset

!unzip gdrive/MyDrive/Possibilities\ Hackathon/Round\ 2/data_round_2.zip

Archive:  gdrive/MyDrive/Possibilities Hackathon/Round 2/data_round_2.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        
  inflating: dataset/sample_submission.csv  


<a id='importing-relevant-data'></a>
## Importing Relevant Data

In [ ]:
# Loading the data
df = pd.read_csv('dataset/train.csv')
df.head()

,index,serviced_from_city,customer_locality,customer,transaction_date,brand,category,product,ordered_qty,ordered_amount
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000


In [ ]:
df_test = pd.read_csv('dataset/test.csv')
df_test.head()

,customer,transaction_date,serviced_from_city,customer_locality
0,0bb1603d16,2022-02-09,279f690075,00ef7bae9d
1,6ec718360a,2022-02-09,279f690075,00ef7bae9d
2,70bb518f2e,2022-02-09,279f690075,00ef7bae9d
3,90f7c4feb1,2022-02-09,279f690075,00ef7bae9d
4,8543ca6455,2022-02-09,279f690075,02b0578819


<a id='data-engineering'></a>
## Data Engineering

In this section, we will explore the given dataset to answer some general questions about the dataset. This will lead to cause us to introduce other features into the dataset which will help us to build a machine learning model that will help us to answer our main question.

Let's update the pandas dataframe `df` by renaming some of its columns.

In [ ]:
# Rename the columns 

df.rename(columns={'index':'InvoiceNo', 'customer':'CustomerID', 'ordered_qty': 'Quantity', 'category': 'Description', 'serviced_from_city': 'Country', 'transaction_date': 'InvoiceDate', 'ordered_amount':'Revenue'}, inplace=True)
df_test.rename(columns={'customer':'CustomerID', 'transaction_date':'InvoiceDate', 'serviced_from_city': 'Country'})
df.head()

,InvoiceNo,Country,customer_locality,CustomerID,InvoiceDate,brand,Description,product,Quantity,Revenue
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000


We can check information about the dataframe with the `info` method. 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955390 entries, 0 to 955389
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   InvoiceNo          955390 non-null  int64  
 1   Country            955390 non-null  object 
 2   customer_locality  955390 non-null  object 
 3   CustomerID         955390 non-null  object 
 4   InvoiceDate        955390 non-null  object 
 5   brand              955390 non-null  object 
 6   Description        955390 non-null  object 
 7   product            955390 non-null  object 
 8   Quantity           955390 non-null  int64  
 9   Revenue            955390 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 72.9+ MB


From the output of the `info` method, we can see that the __CustomerID__ and __Description__  columns are likely to have missing values. Also we may want to change the data type of the objects in the __InvoiceDate__ column to proper date objects with the `to_datetime` method. 

Let's count the number of missing values in each column of the dataframe.

In [ ]:
df.isnull().sum()

InvoiceNo            0
Country              0
customer_locality    0
CustomerID           0
InvoiceDate          0
brand                0
Description          0
product              0
Quantity             0
Revenue              0
dtype: int64

As predicted, there are no columns with null values, so we can move ahead with our task.

In [ ]:
df_data = df.copy()
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955390 entries, 0 to 955389
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   InvoiceNo          955390 non-null  int64  
 1   Country            955390 non-null  object 
 2   customer_locality  955390 non-null  object 
 3   CustomerID         955390 non-null  object 
 4   InvoiceDate        955390 non-null  object 
 5   brand              955390 non-null  object 
 6   Description        955390 non-null  object 
 7   product            955390 non-null  object 
 8   Quantity           955390 non-null  int64  
 9   Revenue            955390 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 72.9+ MB


Next, we update the dataframe `df_data` by converting the date field, __InvoiceDate__ to _datetime_ object. One can use the `to_datetime` method to achieve this.

In [ ]:
df_data.InvoiceDate = pd.to_datetime(df_data.InvoiceDate)
df_data.head()

,InvoiceNo,Country,customer_locality,CustomerID,InvoiceDate,brand,Description,product,Quantity,Revenue
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000


In [ ]:
pd.DataFrame(df_data.InvoiceDate.describe())

,InvoiceDate
count,955390
unique,307
top,2021-05-24 00:00:00
freq,5767
first,2021-04-01 00:00:00
last,2022-02-08 00:00:00


From the above, we see that the online purchases made by customers was from 2021-04-01  to 2022-02-08.

In the subsequent cells, we will answer some questions we would like to know from the given dataset.

<a id='exp-qxn-1'></a>
---
> How many online customers are there in the dataset and what is their country of origin?
---

In [ ]:
print('From the dataset, the online retail shop has {} customers \
from {} different countries.'.format(len(df_data.CustomerID.unique()), len(df_data.Country.unique())))

From the dataset, the online retail shop has 6000 customers from 8 different countries.


<a id='exp-qxn-2'></a>
---
> What are the countries that are most represented in the dataset?
---

In [ ]:
ctm_cntry_df = df_data.groupby(['CustomerID', 'Country']).count().reset_index()

ctm_cntry_df = ctm_cntry_df.groupby('Country')['CustomerID'].count().reset_index().sort_values(
    by=['CustomerID'], ascending=False)

# Create a new column, Percentage to calculate the customer representation in percentage
ctm_cntry_df['Percentage']= np.round(ctm_cntry_df.CustomerID / ctm_cntry_df.CustomerID.sum() * 100, 2)

ctm_cntry_df.head(10)

,Country,CustomerID,Percentage
0,279f690075,1018,16.95
5,8421ba6ad9,832,13.85
6,edb4f165a1,774,12.88
3,5a8a0fd53a,773,12.87
2,48181194be,739,12.30
7,f072608736,689,11.47
1,34aff2d7c6,639,10.64
4,66ccb00c2a,543,9.04


The output above, shows the top __8__ countries of that purchase from the online retail shop with the __279f690075__ being the country code with the highest customers. They represent about $16.95\%$ of the entire online customers. 

In the two next cells, we group countries with customer percentage value less than or equal to $0.25$ together and then plot a pie chart. 

In [ ]:
percent_margin = 0.25

# Create a new column, CountryCategory and set values to the corresponding values of the Country column
ctm_cntry_df['CountryCategory'] = ctm_cntry_df.Country

# Set Countries with Percentage less than or equal to percent_margin to 'Other Countries'

ctm_cntry_df.loc[ctm_cntry_df.Percentage <= percent_margin, 'CountryCategory'] = 'Other Countries'

ctm_cntry_df.head(11)

,Country,CustomerID,Percentage,CountryCategory
0,279f690075,1018,16.95,279f690075
5,8421ba6ad9,832,13.85,8421ba6ad9
6,edb4f165a1,774,12.88,edb4f165a1
3,5a8a0fd53a,773,12.87,5a8a0fd53a
2,48181194be,739,12.30,48181194be
7,f072608736,689,11.47,f072608736
1,34aff2d7c6,639,10.64,34aff2d7c6
4,66ccb00c2a,543,9.04,66ccb00c2a


In [ ]:

bar_fig = px.bar(ctm_cntry_df, 
                 x="CountryCategory", y="Percentage", color="CountryCategory", title="Customer Country Count in Percentage",)

bar_fig.update_layout(title_x=0, 
                      legend_title="Countries Represented", 
                      legend=dict(orientation="h")
                     )

bar_fig.show(config={'displaylogo': False})

<a id='exp-qxn-3'></a>
> Calculate the revenue that was made in each month and what is the percentage revenue based on countries?
---

In [ ]:
df_data['InvoiceYearMonth'] = df_data['InvoiceDate'].map(lambda date: 100*date.year + date.month)
df_data.head()

,InvoiceNo,Country,customer_locality,CustomerID,InvoiceDate,brand,Description,product,Quantity,Revenue,InvoiceYearMonth
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284,202112
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852,202112
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000,202110


In [ ]:
ctm_revenue = df_data.groupby('InvoiceYearMonth').Revenue.sum().reset_index()
ctm_revenue.head()

,InvoiceYearMonth,Revenue
0,202104,5.529194e+07
1,202105,5.601835e+07
2,202106,5.510598e+07
3,202107,5.678650e+07
4,202108,5.845658e+07


In [ ]:
pd.DataFrame(ctm_revenue.Revenue.describe())

,Revenue
count,1.100000e+01
mean,5.937272e+07
std,1.468758e+07
min,2.739122e+07
25%,5.519896e+07
50%,5.678650e+07
75%,6.473067e+07
max,8.870399e+07


In [ ]:
# plot line plot
line_fig = px.line(ctm_revenue, 
                   x = "InvoiceYearMonth", 
                   y = "Revenue", 
                   title = "Montly Revenue from Dec. 2009 to Dec. 2011", 
                   markers=True
                  )

line_fig.update_layout(title_x=0.5, 
                       showlegend=False, 
                       xaxis={"type": "category"}, 
                       xaxis_title="Invoice Year-Month", 
                       yaxis_title="Monthly Revenue"
                      )

line_fig.show(config={'displaylogo': False})

From the graph figure above, we observe that the online retail shop made the highest sales in November 2010, followed by __January 2022__. Also, one can observe that, monthly sales rises after November 2021 and falls after January 2022. 

Next, explore the percentage revenue generated by the retail shop based on the countries their customers reside.

In [ ]:
cntry_revenue_df = df_data.groupby(['Country']).Revenue.sum().reset_index().sort_values(by=['Revenue'], 
                                                                                        ascending=False)

cntry_revenue_df['Percentage'] = np.round(cntry_revenue_df.Revenue / cntry_revenue_df.Revenue.sum() * 100, 2)

cntry_revenue_df.head(5)

,Country,Revenue,Percentage
3,5a8a0fd53a,1.237536e+08,18.95
6,edb4f165a1,1.061656e+08,16.26
0,279f690075,9.216424e+07,14.11
2,48181194be,8.259242e+07,12.65
5,8421ba6ad9,8.152901e+07,12.48


From the output above, the top __5__ country codes with respect to revenue generated are:
 * 5a8a0fd53a
 * edb4f165a1
 * 279f690075 
 * 48181194be
 * 8421ba6ad9

with the __5a8a0fd53a__ recording the highest in percentage $(82.93\%)$. 

In the next two cells, we update `cntry_revenue_df` with a new 

In the two next cells, we group countries with revenue percentage value less than or equal to $0.25$ together and then plot a bargraph. 

In [ ]:
percent_margin = 0.25

# Create a new column, CountryCategory and set values to the corresponding values of the Country column
cntry_revenue_df['CountryCategory'] = cntry_revenue_df.Country

# Set Countries with Percentage less than or equal to percent_margin to 'Other Countries'

cntry_revenue_df.loc[cntry_revenue_df.Percentage <= percent_margin, 'CountryCategory'] = 'Other Countries'

cntry_revenue_df.head(11)

,Country,Revenue,Percentage,CountryCategory
3,5a8a0fd53a,1.237536e+08,18.95,5a8a0fd53a
6,edb4f165a1,1.061656e+08,16.26,edb4f165a1
0,279f690075,9.216424e+07,14.11,279f690075
2,48181194be,8.259242e+07,12.65,48181194be
5,8421ba6ad9,8.152901e+07,12.48,8421ba6ad9
1,34aff2d7c6,8.044690e+07,12.32,34aff2d7c6
7,f072608736,7.526442e+07,11.52,f072608736
4,66ccb00c2a,1.118366e+07,1.71,66ccb00c2a


In [ ]:
bar_fig = px.bar(cntry_revenue_df, 
                 x="CountryCategory", 
                 y="Percentage", 
                 title="Country code Revenue in Percentage", 
                 color="CountryCategory"
                )

bar_fig.update_layout(title_x=0, 
                      legend_title="Country Code Represented", 
                      legend=dict(orientation="h")
                     )

bar_fig.show(config={'displaylogo': False})

<a id='rfm_analysis'></a>
# RFM Analysis

In this section we will see some analysis of customers ans bacis RFM analysis.
RFM analysis: is a marketing technique used to quantitatively rank and group customers based on the `recency, frequency and monetary` total of their recent transactions to `identify the best customers and perform targeted marketing campaigns`.


## Top Customers
Calculate how often did particular customer make purchase during the year

In [ ]:
df_data['freq'] = df_data.groupby('CustomerID')['CustomerID'].transform('count')

In [ ]:
# Calculate the top 10 customers who brings the highest revenue
customer  = pd.DataFrame(df_data.groupby(['CustomerID'])['Revenue'].sum().sort_values(ascending=False)).reset_index().head(10)

In [ ]:
customer['CustomerID'].tolist()

['4e1067e352',
 '8958eb3622',
 'ef4b40a73b',
 'cea85dc359',
 'aa8a5515cb',
 'de3c096e83',
 '8aa406b6d9',
 'dc686a98bd',
 'f547f07666',
 '11e44afbff']

In [ ]:
top_customer = df_data[df_data['CustomerID'].isin(customer['CustomerID'].tolist())][['CustomerID', 'Country', 'Revenue', 'Quantity', 'freq']]

In [ ]:
# Countries where the top customers resids
top_customer['Country'].unique().tolist()

['f072608736',
 '66ccb00c2a',
 '8421ba6ad9',
 'edb4f165a1',
 '279f690075',
 '48181194be',
 '5a8a0fd53a']

Top country code where the top customers resids:


*   f072608736
*   66ccb00c2a
*   8421ba6ad9
*   edb4f165a1



In [ ]:
df_rfm = df_data
df_rfm.head()

,InvoiceNo,Country,customer_locality,CustomerID,InvoiceDate,brand,Description,product,Quantity,Revenue,InvoiceYearMonth,freq
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104,346
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104,362
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284,202112,292
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852,202112,380
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000,202110,411


In [ ]:
print(df_rfm['InvoiceDate'].min())
print(df_rfm['InvoiceDate'].max())

2021-04-01 00:00:00
2022-02-08 00:00:00


In [ ]:
import datetime as dt
presence = dt.datetime(2022, 2, 9)
df_rfm['InvoiceDate'] = pd.to_datetime(df_rfm['InvoiceDate'])

# Calculate recency, frequency and monetary values:

`Recency (R) as days since last purchase`: How many days ago was their last purchase? Deduct most recent purchase date from today to calculate the recency value. 1 day ago? 14 days ago? 500 days ago?

`Frequency (F) as total number of transactions`: How many times has the customer purchased from our store? For example, if someone placed 10 orders over a period of time, their frequency is 10.

`Monetary (M) as total money spent`: How many $$ (or whatever is your currency of calculation) has this customer spent? Again limit to last two years – or take all time. Simply total up the money from all transactions to get the M value.

In [ ]:
rfm = df_rfm.groupby('CustomerID').agg({'InvoiceDate': lambda x: (presence - x.max()).days, 'InvoiceNo': lambda x: len(x), 'Revenue': lambda x: x.sum()})
rfm['InvoiceDate'] = rfm['InvoiceDate'].astype(int)
rfm.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Revenue': 'monetary_value'}, inplace=True)

In [ ]:
rfm.head()


,recency,frequency,monetary_value
CustomerID,,,
0,11,256,166151.863029
001d8d7b71,38,105,18349.744268
0027c457ac,5,243,89006.737944
00342a58e8,258,45,5263.313382
003e8e240d,7,249,738456.290378


In [ ]:
# Define quartiles for rfm score:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [ ]:
rfm_segments= rfm


In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
rfm_segments['R_Quartile'] = rfm_segments['recency'].apply(RClass, args=('recency',quantiles,))
rfm_segments['F_Quartile'] = rfm_segments['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfm_segments['M_Quartile'] = rfm_segments['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [ ]:
rfm_segments['RFMClass'] = rfm_segments.R_Quartile.map(str) \
                            + rfm_segments.F_Quartile.map(str) \
                            + rfm_segments.M_Quartile.map(str)

In [ ]:
rfm_segments.head()


,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,
0,11,256,166151.863029,3,1,1,311
001d8d7b71,38,105,18349.744268,4,3,4,434
0027c457ac,5,243,89006.737944,2,1,2,212
00342a58e8,258,45,5263.313382,4,4,4,444
003e8e240d,7,249,738456.290378,3,1,1,311


In [ ]:
rfm_segments['RFMClass'].unique()


array(['311', '434', '212', '444', '412', '244', '321', '134', '133',
       '344', '121', '433', '233', '111', '144', '221', '123', '223',
       '132', '243', '231', '322', '222', '242', '312', '232', '443',
       '313', '112', '131', '331', '122', '422', '211', '432', '323',
       '423', '333', '421', '343', '213', '113', '332', '442', '143',
       '334', '142', '431', '234', '342', '324', '411', '124', '224',
       '424', '341', '413', '241', '441', '141', '114'], dtype=object)

In [ ]:
print("Champions: ",len(rfm_segments[rfm_segments['RFMClass']=='111']), "(", round(len(rfm_segments[rfm_segments['RFMClass']=='111'])/len(rfm_segments)*100,2), "%)") 
print('Loyal Customers: ',len(rfm_segments[rfm_segments['F_Quartile']==1]), "(", round(len(rfm_segments[rfm_segments['F_Quartile']==1])/len(rfm_segments)*100,2), "%)")
print("Big Spenders: ",len(rfm_segments[rfm_segments['M_Quartile']==1]), "(", round(len(rfm_segments[rfm_segments['M_Quartile']==1])/len(rfm_segments)*100,2), "%)")
print('About to Sleep: ', len(rfm_segments[rfm_segments['RFMClass']=='322']), "(", round(len(rfm_segments[rfm_segments['RFMClass']=='322'])/len(rfm_segments)*100,2), "%)")
print('Lost Customers: ',len(rfm_segments[rfm_segments['RFMClass']=='444']), "(", round(len(rfm_segments[rfm_segments['RFMClass']=='444'])/len(rfm_segments)*100,2), "%)")

Champions:  287 ( 4.78 %)
Loyal Customers:  1491 ( 24.85 %)
Big Spenders:  1500 ( 25.0 %)
About to Sleep:  123 ( 2.05 %)
Lost Customers:  759 ( 12.65 %)


Form the above calculations we can say that there are total `24.85% of loyal customers` and `25.0% of big spenders`.

<a id='predicting-customer-purchase'></a>
## Predicting Customer Purchase 

The goal of this section is to come up with a make a model using the given dataframe `df_data`, to estimate if a given customer will buy something again from the online shop in the next quarter. 

The dataframe is split into two. 

* The first sub-dataframe assigned to the Python variable `ctm_bhvr_dt` contains purchases made by customers from `01-04-2021` to `30-08-2022`. This dataframe will be used to study the behavioural purchases of the online customers. 

* The second sub-dataframe assigned to the Python variable `ctm_next_quarter` will be used to study the behavioural purchases of the customers in the next quarter. That is, from `01-02-2022` to `30-11-2012`.

In [ ]:
df_data.head()

,InvoiceNo,Country,customer_locality,CustomerID,InvoiceDate,brand,Description,product,Quantity,Revenue,InvoiceYearMonth,freq
0,3,f072608736,b4a2215bbe,8352843b0e,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104,346
1,4,f072608736,10c1ce7c59,765c513a35,2021-04-09,a63d2efcb1,Mehendi,Nisha Black Mehendi @100 Linkage,1,56.350000,202104,362
2,5,f072608736,4b1ebbdcde,68843f3124,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,6,205.714284,202112,292
3,6,f072608736,71b1924106,9e8c1232f5,2021-12-11,a4c168f27a,Meat Masala,Suhana Meat Masala @5 Linkage,18,617.142852,202112,380
4,7,f072608736,88e5b010b1,4829d8ecb1,2021-10-23,0d55b07506,Snacks,Yellow Diamond Chips Plain Salted @5 Linkage,4,231.560000,202110,411


In [ ]:
ctm_bhvr_dt = df_data[(df_data.InvoiceDate < pd.Timestamp(2022,2,1)) & 
      (df_data.InvoiceDate >= pd.Timestamp(2021,4,1))].reset_index(drop=True)


ctm_next_quarter = df_data[(df_data.InvoiceDate < pd.Timestamp(2022,12,1)) & 
      (df_data.InvoiceDate >= pd.Timestamp(2022,2,1))].reset_index(drop=True)

In [ ]:
# Get the distinct customers in the dataframe ctm_bhvr_dt
ctm_dt = pd.DataFrame(ctm_bhvr_dt['CustomerID'].unique())

# Rename the column to CustomerID.
ctm_dt.columns = ['CustomerID']

ctm_dt.head()

,CustomerID
0,8352843b0e
1,765c513a35
2,68843f3124
3,9e8c1232f5
4,4829d8ecb1


Let's find the first purchase made by each customer in the next quarter. 

In [ ]:
# Create a dataframe with CustomerID and customers first purchase 
# date in ctm_next_quarter
ctm_1st_purchase_in_next_quarter = ctm_next_quarter.groupby('CustomerID').InvoiceDate.min().reset_index()
ctm_1st_purchase_in_next_quarter.columns = ['CustomerID','MinPurchaseDate']
ctm_1st_purchase_in_next_quarter.head()

,CustomerID,MinPurchaseDate
0,0027c457ac,2022-02-04
1,003e8e240d,2022-02-02
2,0069418aec,2022-02-04
3,007e180180,2022-02-01
4,0087fdc687,2022-02-05


Let's find the last purchase made by each customer in the dataframe `ctm_bhvr_dt`.

In [ ]:
ctm_last_purchase_bhvr_dt = ctm_bhvr_dt.groupby('CustomerID').InvoiceDate.max().reset_index()
ctm_last_purchase_bhvr_dt.columns = ['CustomerID','MaxPurchaseDate']
ctm_last_purchase_bhvr_dt.head()

,CustomerID,MaxPurchaseDate
0,0,2022-01-29
1,001d8d7b71,2022-01-02
2,0027c457ac,2022-01-29
3,00342a58e8,2021-05-27
4,003e8e240d,2022-01-26


Let's merge the two dataframes `ctm_last_purchase_bhvr_dt` and `ctm_1st_purchase_in_next_quarter`.

In [ ]:
# Merge two dataframes ctm_last_purchase_bhvr_dt and ctm_1st_purchase_in_next_quarter
ctm_purchase_dates = pd.merge(ctm_last_purchase_bhvr_dt, ctm_1st_purchase_in_next_quarter, on='CustomerID', 
                              how='left')
ctm_purchase_dates.head()

,CustomerID,MaxPurchaseDate,MinPurchaseDate
0,0,2022-01-29,NaT
1,001d8d7b71,2022-01-02,NaT
2,0027c457ac,2022-01-29,2022-02-04
3,00342a58e8,2021-05-27,NaT
4,003e8e240d,2022-01-26,2022-02-02


Let's calculate the time difference in days between customer's last purchase in the dataframe `ctm_last_purchase_bhvr_dt` and the first purchase in the dataframe `ctm_1st_purchase_in_next_quarter`.

In [ ]:
ctm_purchase_dates['NextPurchaseDay'] = (ctm_purchase_dates['MinPurchaseDate'] - ctm_purchase_dates['MaxPurchaseDate']).dt.days

ctm_purchase_dates.head()

,CustomerID,MaxPurchaseDate,MinPurchaseDate,NextPurchaseDay
0,0,2022-01-29,NaT,NaN
1,001d8d7b71,2022-01-02,NaT,NaN
2,0027c457ac,2022-01-29,2022-02-04,6.0
3,00342a58e8,2021-05-27,NaT,NaN
4,003e8e240d,2022-01-26,2022-02-02,7.0


In [ ]:
# merge with ctm_dt 
ctm_dt = pd.merge(ctm_dt, ctm_purchase_dates[['CustomerID','NextPurchaseDay']], on='CustomerID', how='left')
ctm_dt.head()

,CustomerID,NextPurchaseDay
0,8352843b0e,7.0
1,765c513a35,13.0
2,68843f3124,8.0
3,9e8c1232f5,9.0
4,4829d8ecb1,7.0


Update the dataframe `ctm_dt` by filling all missing values with $9999$.

In [ ]:
ctm_dt = ctm_dt.fillna(9999)
ctm_dt.head()

,CustomerID,NextPurchaseDay
0,8352843b0e,7.0
1,765c513a35,13.0
2,68843f3124,8.0
3,9e8c1232f5,9.0
4,4829d8ecb1,7.0


Next, we will define some features and add them to the dataframe `ctm_dt` to build our machine learning model. We will use the Recency - Frequency - Monetary Value segmentation method. That is, we will put the customers into groups based on the following: 

* __Recency__: Customers purchase behaviour based on their most recent purchase date and how many days they have been inactive since their last purchase.

* __Frequency__: Customers purchase behaviour based on the number of times they buy from the online retail shop.

* __Monetary Value__/__Revenue__: Customers purchase behaviour based the revenue they generate.

After we will apply *K-means* clustering to assign customers a score to each of the features.

<a id='recency'></a>
#### Recency 

Let's find the most recent purchase date of each customer and see how many days they have been inactive. Afterwards, we can apply *K-means* clustering to assign customers a recency score.

In [ ]:
ctm_max_purchase = ctm_bhvr_dt.groupby('CustomerID').InvoiceDate.max().reset_index()
ctm_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
ctm_max_purchase.head()

,CustomerID,MaxPurchaseDate
0,0,2022-01-29
1,001d8d7b71,2022-01-02
2,0027c457ac,2022-01-29
3,00342a58e8,2021-05-27
4,003e8e240d,2022-01-26


In [ ]:
# Find the recency in days 
ctm_max_purchase['Recency'] = (ctm_max_purchase['MaxPurchaseDate'].max() - ctm_max_purchase['MaxPurchaseDate']).dt.days

# Merge the dataframes ctm_dt and ctm_max_purchase[['CustomerID', 'Recency']] on the CustomerID column.
ctm_dt = pd.merge(ctm_dt, ctm_max_purchase[['CustomerID', 'Recency']], on='CustomerID')
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency
0,8352843b0e,7.0,3
1,765c513a35,13.0,9
2,68843f3124,8.0,1
3,9e8c1232f5,9.0,2
4,4829d8ecb1,7.0,2


In [ ]:
pd.DataFrame(ctm_dt.Recency.describe())

,Recency
count,5945.000000
mean,34.764844
std,74.577799
min,0.000000
25%,3.000000
50%,5.000000
75%,14.000000
max,305.000000


The mean _Recency_ is approximately $35$ days whiles the median is $5$ days.

In [ ]:
# plot histogram
hist_fig = px.histogram(ctm_dt, 
                        x="Recency", 
                        title="Customers Recency in Days"
                       )

hist_fig.update_layout(title_x=0.5, 
                       xaxis_title="Recency in groups of 20 days", 
                       yaxis_title="Number of Customers"
                      )

hist_fig.show(config={'displaylogo': False})

Next we will apply _K-means_ clustering to assign a recency score. However, we need to know how many clusters in order to use the _K-means_ algorithm. We will apply _Elbow Method_ to determine how many clusters we will need. The _Elbow Method_ simply tells the optimal cluster number for optimal inertia.

In [ ]:
my_dict={}
ctm_recency = ctm_dt[['Recency']]
for idx in range(1, 10):
    kmeans = KMeans(n_clusters=idx, max_iter=1000).fit(ctm_recency)
    ctm_recency["clusters"] = kmeans.labels_
    my_dict[idx] = kmeans.inertia_ 

line_fig = px.line(x=list(my_dict.keys()), 
                   y=list(my_dict.values()), 
                   markers=True
                  )

line_fig.update_layout(title_x=0, 
                       xaxis_title="Number of cluster", 
                       yaxis_title=""
                      )

line_fig.show(config={'displaylogo': False})

From the Figure above, $2$ seem to be the optimal one.

In [ ]:
number_of_clusters = 2

Let's build a $2$ clusters for recency and add it to dataframe, `ctm_dt`.

In [ ]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Recency']])
ctm_dt['RecencyCluster'] = kmeans.predict(ctm_dt[['Recency']])
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster
0,8352843b0e,7.0,3,0
1,765c513a35,13.0,9,0
2,68843f3124,8.0,1,0
3,9e8c1232f5,9.0,2,0
4,4829d8ecb1,7.0,2,0


In [ ]:
def order_cluster(df, target_field_name, cluster_field_name, ascending):
    """
    INPUT:
        - df                  - pandas DataFrame
        - target_field_name   - str - A column in the pandas DataFrame df
        - cluster_field_name  - str - Expected to be a column in the pandas DataFrame df
        - ascending           - Boolean
        
    OUTPUT:
        - df_final            - pandas DataFrame with target_field_name and cluster_field_name as columns
    
    """
    # Add the string "new_" to cluster_field_name
    new_cluster_field_name = "new_" + cluster_field_name
    
    # Create a new dataframe by grouping the input dataframe by cluster_field_name and extract target_field_name 
    # and find the mean
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    
    # Sort the new dataframe df_new, by target_field_name in descending order
    df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    
    # Create a new column in df_new with column name index and assign it values to df_new.index
    df_new["index"] = df_new.index
    
    # Create a new dataframe by merging input dataframe df and part of the columns of df_new based on 
    # cluster_field_name
    df_final = pd.merge(df, df_new[[cluster_field_name, "index"]], on=cluster_field_name)
    
    # Update the dataframe df_final by deleting the column cluster_field_name
    df_final = df_final.drop([cluster_field_name], axis=1)
    
    # Rename the column index to cluster_field_name
    df_final = df_final.rename(columns={"index": cluster_field_name})
    
    return df_final

In [ ]:
ctm_dt = order_cluster(ctm_dt, 'Recency', 'RecencyCluster', False)
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster
0,8352843b0e,7.0,3,1
1,765c513a35,13.0,9,1
2,68843f3124,8.0,1,1
3,9e8c1232f5,9.0,2,1
4,4829d8ecb1,7.0,2,1


In [ ]:
#print cluster characteristics
ctm_dt.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,667.0,240.101949,24.796259,137.0,241.0,244.0,247.0,305.0
1,5278.0,8.815650,13.580211,0.0,2.0,5.0,9.0,123.0


Observe from the above that, $1$ covers the most recent customers whereas $0$ has the most inactive customers.

<a id='frequency'></a>
#### Frequency 

Next, we will find customers purchase behaviour based on the number of times they buy from the online retail shop. That is, the total number of orders by each customer.

In [ ]:
#get order counts for each user and create a dataframe with it
ctm_frequency = df_data.groupby('CustomerID').InvoiceDate.count().reset_index()
ctm_frequency.columns = ['CustomerID','Frequency']

In [ ]:
#add this data to our main ctm_dt
ctm_dt = pd.merge(ctm_dt, ctm_frequency, on='CustomerID')

ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency
0,8352843b0e,7.0,3,1,346
1,765c513a35,13.0,9,1,362
2,68843f3124,8.0,1,1,292
3,9e8c1232f5,9.0,2,1,380
4,4829d8ecb1,7.0,2,1,411


In [ ]:
pd.DataFrame(ctm_dt.Frequency.describe())

,Frequency
count,5945.000000
mean,160.685618
std,125.093319
min,1.000000
25%,59.000000
50%,150.000000
75%,225.000000
max,1146.000000


In [ ]:
# plot histogram
hist_fig = px.histogram(x=ctm_dt.query('Frequency < 1200')['Frequency'], 
                        title="Customers with Purchase Frequency less than 1200" 
                       )

hist_fig.update_layout(title_x=0.5, 
                       xaxis_title="Customer Frequency Purchase in groups of 20", 
                       yaxis_title="Number of Customers"
                      )

hist_fig.show(config={'displaylogo': False})

In [ ]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Frequency']])
ctm_dt['FrequencyCluster'] = kmeans.predict(ctm_dt[['Frequency']])

In [ ]:
ctm_dt = order_cluster(ctm_dt, 'Frequency', 'FrequencyCluster', False)
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster
0,8352843b0e,7.0,3,1,346,0
1,765c513a35,13.0,9,1,362,0
2,68843f3124,8.0,1,1,292,0
3,9e8c1232f5,9.0,2,1,380,0
4,4829d8ecb1,7.0,2,1,411,0


In [ ]:
#see details of each cluster
ctm_dt.groupby('FrequencyCluster')['Frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,2262.0,283.466844,105.762669,185.0,212.0,251.0,320.0,1146.0
1,3683.0,85.276677,58.681838,1.0,30.0,83.0,140.0,184.0


As it was for the case of the Recency, higher frequency number means better customers.

<a id='revenue'></a>
#### Revenue

In [ ]:
ctm_revenue = df_data.groupby('CustomerID').Revenue.sum().reset_index()

In [ ]:
#merge it with our ctm_dt
ctm_dt = pd.merge(ctm_dt, ctm_revenue, on='CustomerID')
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue
0,8352843b0e,7.0,3,1,346,0,177374.978313
1,765c513a35,13.0,9,1,362,0,181102.871753
2,68843f3124,8.0,1,1,292,0,97728.043213
3,9e8c1232f5,9.0,2,1,380,0,74135.977864
4,4829d8ecb1,7.0,2,1,411,0,105388.984946


In [ ]:
# plot histogram
hist_fig = px.histogram(x=ctm_dt.query('Revenue < 10000')['Revenue'], 
                        title="Customers with Monetary Value below 10000" 
                       )

hist_fig.update_layout(title_x=0.5, 
                       xaxis_title="Customers Revenue", 
                       yaxis_title="Number of Customers"
                      )

hist_fig.show(config={'displaylogo': False})

In [ ]:
#apply clustering
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Revenue']])
ctm_dt['RevenueCluster'] = kmeans.predict(ctm_dt[['Revenue']])

In [ ]:
#order the cluster numbers
ctm_dt = order_cluster(ctm_dt, 'Revenue', 'RevenueCluster', True)
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster
0,8352843b0e,7.0,3,1,346,0,177374.978313,0
1,765c513a35,13.0,9,1,362,0,181102.871753,0
2,68843f3124,8.0,1,1,292,0,97728.043213,0
3,9e8c1232f5,9.0,2,1,380,0,74135.977864,0
4,4829d8ecb1,7.0,2,1,411,0,105388.984946,0


In [ ]:
#show details of the dataframe
ctm_dt.groupby('RevenueCluster')['Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,4816.0,64590.737321,51387.892014,8.882730,19240.916705,54697.829861,102904.686633,1.831113e+05
1,1129.0,302907.516039,137049.517721,183334.398832,217044.455549,263440.031059,341592.295385,2.405535e+06


<a id='overall-score'></a>
#### Overall Score 

Finally we sum all the cluster features

In [ ]:
#calculate overall score and use mean() to see details
ctm_dt['OverallScore'] = ctm_dt['RecencyCluster'] + ctm_dt['FrequencyCluster'] + ctm_dt['RevenueCluster']
ctm_dt.groupby('OverallScore')['Recency','Frequency','Revenue'].mean()

,Recency,Frequency,Revenue
OverallScore,,,
0,241.000000,187.000000,32031.985255
1,78.937441,203.894638,76948.164490
2,10.419198,135.187106,114286.814778
3,6.865854,151.847561,276118.180731


The scoring above clearly shows us that customers with score $8$ are our best customers whereas those who score $3$ are the worst.

In [ ]:
ctm_dt['Segment'] = 'Low-Value'
ctm_dt.loc[ctm_dt['OverallScore'] > 1, 'Segment'] = 'Mid-Value'
ctm_dt.loc[ctm_dt['OverallScore'] > 3, 'Segment'] = 'High-Value'

In [ ]:
ctm_dt.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,8352843b0e,7.0,3,1,346,0,177374.978313,0,1,Low-Value
1,765c513a35,13.0,9,1,362,0,181102.871753,0,1,Low-Value
2,68843f3124,8.0,1,1,292,0,97728.043213,0,1,Low-Value
3,9e8c1232f5,9.0,2,1,380,0,74135.977864,0,1,Low-Value
4,4829d8ecb1,7.0,2,1,411,0,105388.984946,0,1,Low-Value


In [ ]:
#A scatter plot of Revenue verses Frequency

ctm_graph = ctm_dt.query("Revenue < 50000 and Frequency < 2000")


plot_data = [
    go.Scatter(
        x=ctm_graph.query("Segment == 'Low-Value'")['Frequency'],
        y=ctm_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low-Value',
        marker=dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),
        
    go.Scatter(
        x=ctm_graph.query("Segment == 'Mid-Value'")['Frequency'],
        y=ctm_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),
        
    go.Scatter(
        x=ctm_graph.query("Segment == 'High-Value'")['Frequency'],
        y=ctm_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Frequency"},
        title='Segments', 
        title_x = 0.5
    )

fig = go.Figure(
        data=plot_data, 
        layout=plot_layout
    )

#pyoff.iplot(fig)
fig.show(config={'displaylogo': False})

In [ ]:
# Scatter Plot of Revenue verses Recency

ctm_graph = ctm_dt.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=ctm_graph.query("Segment == 'Low-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low-Value',
        marker=dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),
        
    go.Scatter(
        x=ctm_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),
        
    go.Scatter(
        x=ctm_graph.query("Segment == 'High-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Recency"},
        title='Segments', 
        title_x = 0.5
    )

fig = go.Figure(
        data=plot_data, 
        layout=plot_layout
    )

#pyoff.iplot(fig)
fig.show(config={'displaylogo': False})

In [ ]:
# Scatter Plot of Revenue verses Frequency

ctm_graph = ctm_dt.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=ctm_graph.query("Segment == 'Low-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'Low-Value'")['Frequency'],
        mode='markers',
        name='Low-Value',
        marker= dict(
            size=7,
            line=dict(width=1),
            color='blue',
            opacity=0.8
           )
    ),
    
    go.Scatter(
        x=ctm_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'Mid-Value'")['Frequency'],
        mode='markers',
        name='Mid-Value',
        marker=dict(
            size=9,
            line=dict(width=1),
            color='green',
            opacity=0.5
           )
    ),
    
    go.Scatter(
        x=ctm_graph.query("Segment == 'High-Value'")['Recency'],
        y=ctm_graph.query("Segment == 'High-Value'")['Frequency'],
        mode='markers',
        name='High-Value',
        marker=dict(
            size=11,
            line=dict(width=1),
            color='red',
            opacity=0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Frequency"},
        xaxis= {'title': "Recency"},
        title='Segments', 
        title_x = 0.5
    )

fig = go.Figure(
        data=plot_data, 
        layout=plot_layout
    )

#pyoff.iplot(fig)
fig.show(config={'displaylogo': False})

Let us create a copy of the dataframe `ctm_dt` and apply the method `get_dummies` to it so as to convert all categorical column `Segment` to indicator variables. 

In [ ]:
#create ctm_class as a copy of ctm_dt before applying get_dummies
ctm_class = ctm_dt.copy()
ctm_class = pd.get_dummies(ctm_class)
ctm_class.head()

,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,CustomerID_0,CustomerID_001d8d7b71,...,CustomerID_ff883dd557,CustomerID_ff88fdee16,CustomerID_ff963d7ae4,CustomerID_ffb69f75fa,CustomerID_ffbf0a828f,CustomerID_ffc548c1a6,CustomerID_ffea856654,CustomerID_ffefc6b326,Segment_Low-Value,Segment_Mid-Value
0,7.0,3,1,346,0,177374.978313,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,13.0,9,1,362,0,181102.871753,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,8.0,1,1,292,0,97728.043213,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9.0,2,1,380,0,74135.977864,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,7.0,2,1,411,0,105388.984946,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


Since our goal is to estimate whether a customer will make a purchase in the next quarter, we will create a new column `NextPurchaseDayRange` with values as either $1$ or $0$ defined as follows:  
* If the value is $1$, then it indicates that the customer will buy something in the next quarter, i.e., $90$ days from his or her last purchase. 
* The value $0$ indicates that the customer will buy something in more than $90$ days from his or her last purchase.

In [ ]:
ctm_class['NextPurchaseDayRange'] = 1  ## less than 3 months
ctm_class.loc[ctm_class.NextPurchaseDay>90,'NextPurchaseDayRange'] = 0 # more than 3 months
ctm_class.head()

,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,CustomerID_0,CustomerID_001d8d7b71,...,CustomerID_ff88fdee16,CustomerID_ff963d7ae4,CustomerID_ffb69f75fa,CustomerID_ffbf0a828f,CustomerID_ffc548c1a6,CustomerID_ffea856654,CustomerID_ffefc6b326,Segment_Low-Value,Segment_Mid-Value,NextPurchaseDayRange
0,7.0,3,1,346,0,177374.978313,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
1,13.0,9,1,362,0,181102.871753,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
2,8.0,1,1,292,0,97728.043213,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,9.0,2,1,380,0,74135.977864,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,7.0,2,1,411,0,105388.984946,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1


Finally in this section, let's see the correlation between our features and label. We can achieve this by applying the `corr` method to the dataframe `ctm_dt`.

In [ ]:
corr_matrix = ctm_class[ctm_class.columns].corr()
corr_df = pd.DataFrame(corr_matrix.min())
corr_df.columns = ['MinCorrelationCoeff']
corr_df['MaxCorrelationCoeff'] = corr_matrix[corr_matrix < 1].max()
corr_df

,MinCorrelationCoeff,MaxCorrelationCoeff
NextPurchaseDay,-0.999632,0.541567
Recency,-0.978865,0.541567
RecencyCluster,-0.978865,0.493865
Frequency,-0.769272,0.478594
FrequencyCluster,-0.769272,0.476468
...,...,...
CustomerID_ffea856654,-0.016551,0.028811
CustomerID_ffefc6b326,-0.010233,0.010165
Segment_Low-Value,-1.000000,0.442888
Segment_Mid-Value,-1.000000,0.918014


From the output above, we observe that __Overall Score__ has the highest positive correlation of $0.92$ with __RecencyCluster__ and __Segment_Low-Value__ has the highest negative of -1.00 with __Segment_Mid-Value__. 

We can get a good visualisation of the coefficient matrix below.

In [ ]:
# plt.figure(figsize = (40, 30))
# sns.heatmap(corr_matrix, annot = True, linewidths=0.2, fmt=".2f");

<a id='building-machine-learning-models'></a>
## Building Machine Learning Models

In [ ]:
ctm_class = ctm_class.drop('NextPurchaseDay', axis=1)

In [ ]:
X, y = ctm_class.drop('NextPurchaseDayRange', axis=1), ctm_class.NextPurchaseDayRange
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None, shuffle=True)

In [ ]:
# Create an array of models
models = []
models.append(("xgb.XGBClassifier", xgb.XGBClassifier(eval_metric=["auc","error"])))


In [ ]:
models[0][1]

XGBClassifier(eval_metric=['auc', 'error'])

In [ ]:
from xgboost import XGBClassifier, plot_tree


In [ ]:
# Measuring the metrics of the different models

scorer = MultiScorer({'accuracy'  : (accuracy_score , {}), 
                      'f1_score'  : (f1_score       , {'pos_label': 3, 'average':'macro'}), 
                      'recall'    : (recall_score   , {'pos_label': 3, 'average':'macro'}), 
                      'precision' : (precision_score, {'pos_label': 3, 'average':'macro'})
                     })

# A dictionary for all the distinct models and their respective metrics
model_scores_dict = {'model_name' : [], 
                     'accuracy'   : [], 
                     'f1_score'   : [], 
                     'recall'     : [], 
                     'precision'  : [],
                     'time'       : []
                    }

# For each model name and model in models
for model_name, model in models: 
    
    # Add model_name to model_scores_dict 
    model_scores_dict['model_name'].append(model_name)
    #print(model_name)
    kfold = KFold(n_splits=2, random_state=24, shuffle=True)
    start = time.time()
    dummy = cross_val_score(model, X_train, y_train, cv = kfold, scoring = scorer)
    cv_result = scorer.get_results()
    
    # For each metric in cv_result.keys()
    for metric_name in cv_result.keys():
        # Get the average of cv_result[metric_name]
        average_score = np.average(cv_result[metric_name])
        # Update model_scores_dict with average_score for model_name
        model_scores_dict[metric_name].append(average_score)
        #print('%s : %f' %(metric_name, average_score))

    model_scores_dict['time'].append((time.time() - start))
    #print('time : ', time.time() - start, '\n\n')
        
model_score_df = pd.DataFrame(model_scores_dict).set_index("model_name")
model_score_df.sort_values(by=["accuracy", "f1_score", "time"], ascending=False)

,accuracy,f1_score,recall,precision,time
model_name,,,,,
xgb.XGBClassifier,0.83831,0.802223,0.781759,0.853202,60.874786


# Finally the **`Recall Score: 78.17%`**